In [71]:
%load_ext autoreload
%autoreload 2

import os
from os.path import join as pjoin
from tqdm import tqdm

import numpy as np
import pandas as pd

# from dtaidistance import dtw
import pywt

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier 
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost import XGBClassifier

import torch

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
from utils import *
from viz import *

# Hyperparameters

## EDA

In [73]:
PLOT = False

## PREPROCESSING

In [74]:
SMOOTHING_WINDOW = 1 # 1 for no smoothing
DOWNSAMPLE_WINDOW = 1 # 1 for no downsampling

## ML

# Get the data

In [75]:
data_folder = pjoin("data", "MixedShapesRegularTrain")
train_fname = "MixedShapesRegularTrain_TRAIN.tsv"
test_fname = "MixedShapesRegularTrain_TEST.tsv"

train_data = pd.read_csv(pjoin(data_folder, train_fname), sep='\t')

# EDA

In [76]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Columns: 1025 entries, 1 to 1.1120469
dtypes: float64(1024), int64(1)
memory usage: 3.9 MB


In [77]:
train_data.head()

,1,1.1309687,1.1204832,1.1077719,1.0424568,0.96409116,0.91385703,0.9255513,0.90369937,0.91716544,...,0.67574464,0.66714198,0.75205753,0.82026784,0.8957213,0.9829251,1.0491094,1.0931087,1.1025334,1.1120469
0,1,0.830904,0.815222,0.803198,0.742477,0.683805,0.672666,0.673233,0.679851,0.664352,...,0.545351,0.540754,0.603622,0.650685,0.708652,0.771576,0.791044,0.816584,0.819874,0.823568
1,1,0.816833,0.810941,0.772436,0.734036,0.695747,0.657560,0.617705,0.579676,0.541709,...,0.677791,0.709058,0.710436,0.729174,0.743558,0.785415,0.802844,0.805283,0.807960,0.810870
2,1,1.942775,1.928545,1.903324,1.879062,1.812798,1.746591,1.675465,1.611625,1.548000,...,1.660595,1.701224,1.748952,1.794030,1.845204,1.864267,1.860472,1.909628,1.958690,1.944351
3,1,1.873812,1.863549,1.814253,1.779266,1.766919,1.735888,1.675732,1.699785,1.720428,...,1.818965,1.854731,1.890964,1.918501,1.903307,1.910518,1.895921,1.881597,1.873381,1.898326
4,1,2.314335,2.302548,2.283370,2.246295,2.189444,2.150547,2.092030,2.038564,2.015111,...,1.970946,2.020198,2.072746,2.123032,2.172533,2.219684,2.267653,2.315809,2.317153,2.329100


In [78]:
#TODO: should I sort the column names?

**Conclusion: the features and the labels can be seperated by the column dtype**

In [79]:
# splitting the dataset to its features and labels
features = train_data.select_dtypes(include=['float64'])
labels = train_data.select_dtypes(include=['int64'])


In [80]:
# get the labels column name
print(labels.columns)

Index(['1'], dtype='object')


In [81]:
if PLOT:
    # get labels stats
    plt.figure(figsize=(15,3))
    ax = sns.countplot(y="1", data=labels)

    for p in ax.patches:
        ax.annotate(f'{p.get_width()}', (p.get_width()+1, p.get_y()+0.5))

**Conclusion: balanced dataset**

### Something with the column names doesn't make sense. maybe it should be sorted?

In [82]:
y = features.columns.astype("float")
x = np.arange(len(y))

In [83]:
if PLOT:
    fig, axes = plt.subplots(1,2, figsize=(10,5))
    axes[0].plot(x,y)
    axes[1].plot(x,sorted(y))

    axes[0].set_title("Original columns order")
    axes[1].set_title("Sorted columns order")

**Conclusion: Maybe it should be sorted... I should read the papers refferd to in the README.md file to better understand**

## Visualuziation

In [84]:
NUM_SAMPLES_PER_CLASS = 10

In [85]:
if PLOT:
    samples = visualize_dataset(train_data, NUM_SAMPLES_PER_CLASS)

**Conclusion: looks like a clustering \ knn algorithm will be usefull. I've noticed that some classes (e.g. 1) might have multiple sub-clusters**
**Conclusion: looks like a low pass filtering would be usefull**

In [86]:
if PLOT:
    rolling_dataset = pd.concat([labels, features.rolling(SMOOTHING_WINDOW, axis=1).mean()], axis=1)
    rolling_dataset = rolling_dataset.dropna(axis=1)
    rolling_samples = visualize_dataset(rolling_dataset, NUM_SAMPLES_PER_CLASS)

## Data Cleaning

In [87]:
# any dupplicates?
features.duplicated().any()

False

**Conclusion: No**

In [88]:
# any missing values?
features.isna().sum().sum()

0

**Conclusion: No**

In [89]:
# Low pass filtering
features = features.rolling(SMOOTHING_WINDOW, axis=1).mean()
features = features.dropna(axis=1)
    
features.shape

(499, 1024)

In [90]:
# downsampling
features = features[features.columns[::DOWNSAMPLE_WINDOW]]

features.shape

(499, 1024)

In [91]:
if PLOT:
    samples = visualize_dataset(pd.concat([labels, features], axis=1), NUM_SAMPLES_PER_CLASS)

## Remove outliers

# Data splitting

In [22]:
#TODO: do a kfold cross validation split


In [23]:
# splitting to train, val, test
X_train, X_test, y_train, y_test = train_test_split(features, labels, stratify=labels, test_size=0.1)#, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.22)#, random_state=1)


print("Train: ", np.unique(y_train, return_counts = True))
print("Val: ", np.unique(y_val, return_counts = True))
print("Test: ", np.unique(y_test, return_counts = True))

Train:  (array([1, 2, 3, 4, 5], dtype=int64), array([70, 70, 70, 70, 70], dtype=int64))
Val:  (array([1, 2, 3, 4, 5], dtype=int64), array([19, 20, 20, 20, 20], dtype=int64))
Test:  (array([1, 2, 3, 4, 5], dtype=int64), array([10, 10, 10, 10, 10], dtype=int64))


# Preprocessing

**Conclusion: all sequences have the same length, therefore there is no need for special care in this aspect**

In [24]:
#label encoding
# no need for a LabelEncoder(), just substruct 1 from the labels to have it 0-based.

# Feature extraction

# Modeling

## Experiment #1: KNN with DTW

* _

## Testing the DTW metric

In [25]:
from dtaidistance import dtw
# from fastdtw import fastdtw
# from scipy.spatial.distance import euclidean

ModuleNotFoundError: No module named 'dtaidistance'

In [ ]:
dists = [ [ None ]*len(class_names) ]*len(class_names)

# for i in tqdm(range(len(class_names))):
for i in range(len(class_names)):

    for j in range(i, len(class_names)):
        cls_x = X_train.loc[y_train["1"]==class_names[i]]
        cls_y = X_train.loc[y_train["1"]==class_names[j]]
        dists[i][j] = [dtw.distance_fast(cls_x.iloc[x].values, cls_y.iloc[y].values) for x in range(5) for y in range(5)]
#         for x in range(5):
#             for y in range(5):
#                 s1 = cls_x.iloc[x].values
#                 s2 = cls_y.iloc[y].values
#                 print(i,j,x,y)
#                 dists[i][j].append(dtw.distance_fast(s1, s2))


In [ ]:
fig, axes = plt.subplots(len(class_names), len(class_names), figsize=(15,10), sharey=True)

for i in range(len(dists)):
    for j in range(i, len(dists)):
        sns.distplot(x=dists[i][j], ax=axes[i][j])
        if i==0:
            axes[i][j].set_title(f"Class: {cls}")
    

plt.tight_layout()

In [ ]:
dists3 = np.zeros((len(cls1),len(cls1)))
for i in tqdm(range(len(cls2))):
    for j in range(len(cls2)):
        s1 = cls2.iloc[i].values
        s2 = cls2.iloc[j].values
        dists3[i,j] = dtw.distance_fast(s1, s2)
# sns.distplot(dists3)

In [ ]:
sns.heatmap(dists3)

In [ ]:
# nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree', metric=dtw.distance_fast)
cls = KNeighborsClassifier(n_neighbors=1, algorithm='ball_tree', metric=dtw.distance_fast)
cls.fit(X_train, y_train["1"])
y_pred = cls.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

In [ ]:
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

In [ ]:
cls = KNeighborsClassifier(n_neighbors=1, algorithm='ball_tree', metric=dtw.distance_fast)

In [ ]:
# cv_scores = cross_val_score(cls, 
#                             features, 
#                             labels["1"], 
#                             cv=5, # integer, to specify the number of folds in a (Stratified)KFold,
#                             scoring="f1_weighted",
#                             n_jobs=-1,
#                            )
# cv_scores

In [ ]:
grid_params = {
    "n_neighbors": [1,3,5,7,9],
    "weights": ["uniform", "distance"],
    "metric": [dtw.distance_fast, "minkowski"]
}

knn_gs = GridSearchCV(
    KNeighborsClassifier(),
    grid_params,
    verbose=1,
    cv=5,
    n_jobs=-1,
)

In [ ]:
knn_gs_results = gs.fit(features, labels["1"])

In [ ]:
print("best_score: ", knn_gs_results.best_score_)
print("best_estimator_: ", knn_gs_results.best_estimator_)
print("best_params_: ", knn_gs_results.best_params_)

In [ ]:
scaler = preprocessing.MinMaxScaler()
features_scaled = scaler.fit_transform(features)


In [ ]:
knn_gs_results_scaled = knn_gs.fit(features, labels["1"])

## Experiment #2: Feature extraction with DWT + classic ML classifier

In [ ]:
import pywt

In [ ]:
# CWT + CNN
# scales = np.arange(1,128)
# waveletname = 'mexh'
# sample = features.iloc[400]
# time = np.arange(len(sample.index))
# signal = sample.values
# plot_wavelet(time, signal, scales, waveletname=waveletname)

In [ ]:
DECONSTRUCT_LEVEL = 3

coeffs = pywt.wavedec(features.iloc[0], 'db2', level=DECONSTRUCT_LEVEL)
nth_level_approx_coeffs = coeffs[0]
nth_level_approx_coeffs.shape

In [ ]:
cwt_features = []
for i, row in features.iterrows():
    coeffs = pywt.wavedec(row, 'db2', level=DECONSTRUCT_LEVEL)
    nth_level_approx_coeffs = coeffs[0]
    cwt_features.append(nth_level_approx_coeffs)
    
cwt_features = pd.DataFrame(cwt_features, index=features.index)

In [ ]:
grid_params = {
    "max_depth": list(range(2,32,2)),
}

rf_gs = GridSearchCV(
    RandomForestClassifier(),
    grid_params,
    verbose=1,
    cv=5,
    n_jobs=-1,
)

In [ ]:
rf_gs_results = rf_gs.fit(cwt_features, labels["1"])

In [ ]:
print("best_score: ", rf_gs_results.best_score_)
print("best_estimator_: ", rf_gs_results.best_estimator_)
print("best_params_: ", rf_gs_results.best_params_)

In [ ]:
cwt_features_scaled = preprocessing.scale(cwt_features)
# cwt_features_scaled = cwt_features - cwt_features.mean()


In [ ]:
pca = PCA(n_components=2)
pca.fit(cwt_features_scaled)
X = pca.transform(cwt_features_scaled)

In [ ]:
cls1 = X[np.where(labels["1"]==1)[0]]
cls2 = X[np.where(labels["1"]==2)[0]]
cls3 = X[np.where(labels["1"]==3)[0]]
cls4 = X[np.where(labels["1"]==4)[0]]
cls5 = X[np.where(labels["1"]==5)[0]]

In [ ]:
plt.scatter(cls1[:, 0], cls1[:, 1], label=1)
plt.scatter(cls2[:, 0], cls2[:, 1], label=2)
plt.scatter(cls3[:, 0], cls3[:, 1], label=3)
plt.scatter(cls4[:, 0], cls4[:, 1], label=4)
plt.scatter(cls5[:, 0], cls5[:, 1], label=5)
plt.legend()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(cwt_features, labels_1["1"], stratify=labels, test_size=0.2)#, random_state=1)

In [ ]:
labels_1 = labels-1

xgb_model = xgb.XGBClassifier()

D_train = xgb.DMatrix(X_train, label=y_train)
D_val = xgb.DMatrix(X_val, label=y_val)

param = {
    'eta': 0.3, 
    'max_depth': 8,  
    'objective': 'multi:softprob',  
    'num_class': 5} 

# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

steps = 20  # The number of training iterations

model = xgb.train(param, D_train, steps)

y_pred = model.predict(D_val)
best_preds = np.asarray([np.argmax(line) for line in y_pred])

print(classification_report(y_val, best_preds))
print(confusion_matrix(y_val, best_preds))

## Experiment #3: Deep LSTM + FC classifier

In [96]:
from multiprocessing import cpu_count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from deep_ts import LSTMClassifier, Trainer

from utils import *

torch.manual_seed(1)

In [97]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using {device}")

Using cuda


In [98]:
print('Preparing datasets')
trn_ds, val_ds, enc = create_datasets(features, labels["1"])

bs = features.shape[0]
print(f'Creating data loaders with batch size: {bs}')
trn_dl, val_dl = create_loaders(trn_ds, val_ds, bs) #, jobs=cpu_count())

Preparing datasets
Creating data loaders with batch size: 499


In [99]:
for layer_dim in [1,2,3,4,5]:
    for hidden_dim in [512, 256, 128, 64, 32, 16]:
        input_dim = features.shape[1]
#         hidden_dim = 128
#         layer_dim = 1
        output_dim = 5

        n_epochs = 1000
        patience = 100
        lr = 0.0005
        iterations_per_epoch = len(trn_dl)

        model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim)
        model = model.to(device)
        criterion = nn.CrossEntropyLoss()
        opt = torch.optim.RMSprop(model.parameters(), lr=lr)
        sched = CyclicLR(opt, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

        Trainer(model, trn_dl, val_dl, n_epochs, sched, opt, criterion, device, patience)


Start model training
LSTMClassifier(
  (rnn): LSTM(1024, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=512, bias=True)
  (fc_out): Linear(in_features=512, out_features=5, bias=True)
)
Epoch 1 best model saved with f1_score: 59.65%
Epoch 2 best model saved with f1_score: 60.38%
Epoch 3 best model saved with f1_score: 63.74%
Epoch 5 best model saved with f1_score: 70.12%
Epoch 6 best model saved with f1_score: 77.86%
Epoch 7 best model saved with f1_score: 89.78%
Epoch:  10. Loss: 0.3432. F1 score: 79.33%, Acc.: 86.00%
Epoch 15 best model saved with f1_score: 93.02%
Epoch:  20. Loss: 0.2585. F1 score: 86.43%, Acc.: 90.00%
Epoch:  30. Loss: 0.1473. F1 score: 92.80%, Acc.: 94.00%
Epoch 37 best model saved with f1_score: 94.41%
Epoch:  40. Loss: 0.1051. F1 score: 94.28%, Acc.: 95.00%
Epoch 46 best model saved with f1_score: 95.68%
Epoch:  50. Loss: 0.0626. F1 score: 95.68%, Acc.: 96.00%
Epoch:  60. Loss: 0.0449. F1 score: 95.63%, Acc.: 96.00%
Epoch 68 best model saved 

Epoch:  50. Loss: 0.3623. F1 score: 92.80%, Acc.: 94.00%
Epoch 50 best model saved with f1_score: 92.80%
Epoch:  60. Loss: 0.2969. F1 score: 91.53%, Acc.: 93.00%
Epoch:  70. Loss: 0.2496. F1 score: 91.53%, Acc.: 93.00%
Epoch 79 best model saved with f1_score: 92.93%
Epoch:  80. Loss: 0.2108. F1 score: 91.53%, Acc.: 93.00%
Epoch:  90. Loss: 0.1805. F1 score: 92.93%, Acc.: 94.00%
Epoch: 100. Loss: 0.1582. F1 score: 92.93%, Acc.: 94.00%
Epoch: 110. Loss: 0.1377. F1 score: 92.93%, Acc.: 94.00%
Epoch 117 best model saved with f1_score: 93.02%
Epoch: 120. Loss: 0.1207. F1 score: 92.93%, Acc.: 94.00%
Epoch 126 best model saved with f1_score: 94.30%
Epoch: 130. Loss: 0.1074. F1 score: 94.30%, Acc.: 95.00%
Epoch: 140. Loss: 0.0952. F1 score: 95.58%, Acc.: 96.00%
Epoch 140 best model saved with f1_score: 95.58%
Epoch: 150. Loss: 0.0851. F1 score: 94.30%, Acc.: 95.00%
Epoch: 160. Loss: 0.0764. F1 score: 94.30%, Acc.: 95.00%
Epoch: 170. Loss: 0.0683. F1 score: 94.30%, Acc.: 95.00%
Epoch: 180. Loss

Epoch:  40. Loss: 0.1251. F1 score: 95.63%, Acc.: 96.00%
Epoch 40 best model saved with f1_score: 95.63%
Epoch:  50. Loss: 0.1596. F1 score: 95.63%, Acc.: 96.00%
Epoch:  60. Loss: 0.0344. F1 score: 94.29%, Acc.: 95.00%
Epoch 68 best model saved with f1_score: 96.90%
Epoch:  70. Loss: 0.0675. F1 score: 95.50%, Acc.: 96.00%
Epoch:  80. Loss: 0.0162. F1 score: 94.29%, Acc.: 95.00%
Epoch:  90. Loss: 0.0108. F1 score: 95.56%, Acc.: 96.00%
Epoch: 100. Loss: 0.0078. F1 score: 95.56%, Acc.: 96.00%
Epoch: 110. Loss: 0.0059. F1 score: 95.56%, Acc.: 96.00%
Epoch: 120. Loss: 0.0047. F1 score: 95.56%, Acc.: 96.00%
Epoch: 130. Loss: 0.0038. F1 score: 95.56%, Acc.: 96.00%
Epoch: 140. Loss: 0.0031. F1 score: 95.56%, Acc.: 96.00%
Epoch: 150. Loss: 0.0026. F1 score: 95.56%, Acc.: 96.00%
Epoch: 160. Loss: 0.0022. F1 score: 95.56%, Acc.: 96.00%
Early stopping on epoch 168
Start model training
LSTMClassifier(
  (rnn): LSTM(1024, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_feature

Epoch 117 best model saved with f1_score: 74.31%
Epoch: 120. Loss: 0.8350. F1 score: 72.63%, Acc.: 77.00%
Epoch 123 best model saved with f1_score: 74.37%
Epoch: 130. Loss: 0.7845. F1 score: 74.31%, Acc.: 78.00%
Epoch 136 best model saved with f1_score: 75.81%
Epoch 137 best model saved with f1_score: 75.87%
Epoch: 140. Loss: 0.7362. F1 score: 75.81%, Acc.: 79.00%
Epoch 143 best model saved with f1_score: 76.99%
Epoch: 150. Loss: 0.6885. F1 score: 76.99%, Acc.: 80.00%
Epoch: 160. Loss: 0.6434. F1 score: 76.99%, Acc.: 80.00%
Epoch: 170. Loss: 0.5979. F1 score: 76.99%, Acc.: 80.00%
Epoch: 180. Loss: 0.5560. F1 score: 76.99%, Acc.: 80.00%
Epoch: 190. Loss: 0.5145. F1 score: 76.99%, Acc.: 80.00%
Epoch 195 best model saved with f1_score: 77.40%
Epoch: 200. Loss: 0.4743. F1 score: 76.99%, Acc.: 80.00%
Epoch 203 best model saved with f1_score: 78.36%
Epoch: 210. Loss: 0.4369. F1 score: 77.92%, Acc.: 81.00%
Epoch 212 best model saved with f1_score: 78.85%
Epoch 215 best model saved with f1_sco

Epoch 21 best model saved with f1_score: 57.24%
Epoch 22 best model saved with f1_score: 59.25%
Epoch 23 best model saved with f1_score: 60.69%
Epoch 24 best model saved with f1_score: 64.10%
Epoch 25 best model saved with f1_score: 70.67%
Epoch 26 best model saved with f1_score: 72.79%
Epoch 29 best model saved with f1_score: 77.40%
Epoch:  30. Loss: 0.6030. F1 score: 77.85%, Acc.: 83.00%
Epoch 30 best model saved with f1_score: 77.85%
Epoch 32 best model saved with f1_score: 79.43%
Epoch:  40. Loss: 0.4366. F1 score: 78.33%, Acc.: 84.00%
Epoch:  50. Loss: 0.3522. F1 score: 73.07%, Acc.: 83.00%
Epoch:  60. Loss: 0.3045. F1 score: 80.56%, Acc.: 86.00%
Epoch 60 best model saved with f1_score: 80.56%
Epoch 64 best model saved with f1_score: 81.71%
Epoch:  70. Loss: 0.2424. F1 score: 83.87%, Acc.: 88.00%
Epoch 70 best model saved with f1_score: 83.87%
Epoch 76 best model saved with f1_score: 86.44%
Epoch 78 best model saved with f1_score: 88.19%
Epoch:  80. Loss: 0.2027. F1 score: 89.66%,

Epoch: 260. Loss: 0.8502. F1 score: 43.02%, Acc.: 52.00%
Epoch 260 best model saved with f1_score: 43.02%
Epoch 265 best model saved with f1_score: 43.10%
Epoch: 270. Loss: 0.8356. F1 score: 43.02%, Acc.: 52.00%
Epoch: 280. Loss: 0.8199. F1 score: 43.10%, Acc.: 52.00%
Epoch: 290. Loss: 0.7957. F1 score: 42.69%, Acc.: 52.00%
Epoch 295 best model saved with f1_score: 43.10%
Epoch 296 best model saved with f1_score: 43.49%
Epoch 297 best model saved with f1_score: 43.68%
Epoch 298 best model saved with f1_score: 43.81%
Epoch 299 best model saved with f1_score: 43.93%
Epoch: 300. Loss: 0.7720. F1 score: 43.95%, Acc.: 52.00%
Epoch 300 best model saved with f1_score: 43.95%
Epoch 301 best model saved with f1_score: 44.36%
Epoch 302 best model saved with f1_score: 44.54%
Epoch 304 best model saved with f1_score: 46.24%
Epoch 305 best model saved with f1_score: 46.41%
Epoch 309 best model saved with f1_score: 47.74%
Epoch: 310. Loss: 0.7447. F1 score: 49.06%, Acc.: 54.00%
Epoch 310 best model 

Epoch:  80. Loss: 0.0515. F1 score: 90.98%, Acc.: 92.00%
Epoch:  90. Loss: 0.0103. F1 score: 94.61%, Acc.: 95.00%
Epoch: 100. Loss: 0.0031. F1 score: 95.63%, Acc.: 96.00%
Epoch: 110. Loss: 0.0015. F1 score: 95.63%, Acc.: 96.00%
Epoch: 120. Loss: 0.0009. F1 score: 95.63%, Acc.: 96.00%
Epoch: 130. Loss: 0.0007. F1 score: 95.63%, Acc.: 96.00%
Epoch: 140. Loss: 0.0005. F1 score: 95.63%, Acc.: 96.00%
Epoch: 150. Loss: 0.0004. F1 score: 95.63%, Acc.: 96.00%
Epoch: 160. Loss: 0.0003. F1 score: 95.63%, Acc.: 96.00%
Early stopping on epoch 167
Start model training
LSTMClassifier(
  (rnn): LSTM(1024, 256, num_layers=4, batch_first=True)
  (fc): Linear(in_features=256, out_features=256, bias=True)
  (fc_out): Linear(in_features=256, out_features=5, bias=True)
)
Epoch 1 best model saved with f1_score: 4.60%
Epoch 3 best model saved with f1_score: 32.41%
Epoch 4 best model saved with f1_score: 44.45%
Epoch 6 best model saved with f1_score: 48.75%
Epoch 7 best model saved with f1_score: 61.06%
Epoch

Epoch: 350. Loss: 0.0009. F1 score: 87.01%, Acc.: 89.00%
Epoch: 360. Loss: 0.0008. F1 score: 87.01%, Acc.: 89.00%
Epoch: 370. Loss: 0.0008. F1 score: 87.01%, Acc.: 89.00%
Epoch: 380. Loss: 0.0007. F1 score: 87.01%, Acc.: 89.00%
Epoch: 390. Loss: 0.0006. F1 score: 87.01%, Acc.: 89.00%
Early stopping on epoch 395
Start model training
LSTMClassifier(
  (rnn): LSTM(1024, 32, num_layers=4, batch_first=True)
  (fc): Linear(in_features=32, out_features=32, bias=True)
  (fc_out): Linear(in_features=32, out_features=5, bias=True)
)
Epoch 1 best model saved with f1_score: 4.60%
Epoch:  10. Loss: 1.5956. F1 score: 4.60%, Acc.: 13.00%
Epoch 14 best model saved with f1_score: 24.62%
Epoch 16 best model saved with f1_score: 25.28%
Epoch 19 best model saved with f1_score: 42.30%
Epoch:  20. Loss: 1.4929. F1 score: 46.25%, Acc.: 56.00%
Epoch 20 best model saved with f1_score: 46.25%
Epoch 21 best model saved with f1_score: 50.66%
Epoch 22 best model saved with f1_score: 62.44%
Epoch 27 best model save

Epoch 46 best model saved with f1_score: 85.80%
Epoch 47 best model saved with f1_score: 87.53%
Epoch 48 best model saved with f1_score: 89.33%
Epoch:  50. Loss: 0.2584. F1 score: 87.56%, Acc.: 91.00%
Epoch 57 best model saved with f1_score: 91.77%
Epoch:  60. Loss: 0.8079. F1 score: 81.60%, Acc.: 87.00%
Epoch 63 best model saved with f1_score: 93.23%
Epoch:  70. Loss: 0.3227. F1 score: 85.60%, Acc.: 89.00%
Epoch:  80. Loss: 0.0852. F1 score: 92.13%, Acc.: 93.00%
Epoch:  90. Loss: 0.0409. F1 score: 92.19%, Acc.: 93.00%
Epoch: 100. Loss: 0.0234. F1 score: 93.60%, Acc.: 94.00%
Epoch 100 best model saved with f1_score: 93.60%
Epoch: 110. Loss: 0.0245. F1 score: 92.19%, Acc.: 93.00%
Epoch: 120. Loss: 0.0103. F1 score: 92.19%, Acc.: 93.00%
Epoch: 130. Loss: 0.0056. F1 score: 92.19%, Acc.: 93.00%
Epoch: 140. Loss: 0.0035. F1 score: 92.19%, Acc.: 93.00%
Epoch: 150. Loss: 0.0025. F1 score: 92.19%, Acc.: 93.00%
Epoch: 160. Loss: 0.0018. F1 score: 92.19%, Acc.: 93.00%
Epoch: 170. Loss: 0.0014. F

Epoch 38 best model saved with f1_score: 32.42%
Epoch:  40. Loss: 1.0934. F1 score: 31.40%, Acc.: 41.00%
Epoch 43 best model saved with f1_score: 34.38%
Epoch:  50. Loss: 1.0054. F1 score: 34.46%, Acc.: 42.00%
Epoch 50 best model saved with f1_score: 34.46%
Epoch 51 best model saved with f1_score: 35.11%
Epoch:  60. Loss: 0.9738. F1 score: 20.19%, Acc.: 33.00%
Epoch:  70. Loss: 0.9586. F1 score: 20.40%, Acc.: 33.00%
Epoch:  80. Loss: 0.9413. F1 score: 21.88%, Acc.: 29.00%
Epoch:  90. Loss: 0.9012. F1 score: 24.21%, Acc.: 32.00%
Epoch 95 best model saved with f1_score: 35.26%
Epoch 97 best model saved with f1_score: 36.20%
Epoch 99 best model saved with f1_score: 38.99%
Epoch: 100. Loss: 0.8549. F1 score: 32.96%, Acc.: 38.00%
Epoch 101 best model saved with f1_score: 45.26%
Epoch 103 best model saved with f1_score: 47.45%
Epoch 105 best model saved with f1_score: 50.65%
Epoch 107 best model saved with f1_score: 53.24%
Epoch 109 best model saved with f1_score: 58.27%
Epoch: 110. Loss: 0.

In [ ]:
model

In [ ]:
print('The training is finished! Restoring the best model weights')

model.load_state_dict(torch.load('best.pth'))

In [ ]:
model.eval()

In [ ]:
a = preds
a = [a for i in range(10)]

In [ ]:
y_val